<a href="https://colab.research.google.com/github/kiratkJaura/Final_Project03/blob/main/Final_Project03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
project_dir = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data'  #PATH to Project3 Data file
os.chdir(project_dir)

# Install required libraries
!pip install opencv-python-headless ultralytics matplotlib scikit-learn

# Import libraries
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

##  Step 1: Object Masking
image_path = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/motherboard_image.JPEG'  # PATH to Motherboard_image
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresholded = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
)
edges = cv2.Canny(thresholded, 100, 200)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mask = np.zeros_like(gray)
cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
extracted = cv2.bitwise_and(image, image, mask=mask)
extracted_path = 'extracted_motherboard.JPEG'
cv2.imwrite(extracted_path, extracted)
print(f"Extracted image saved to {extracted_path}")

##  Step 2: YOLOv8 Training
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/data.yaml'  #PATH to data.yaml to run YOLOv8
model = YOLO('yolov8n.pt')
model.train(data=dataset_path, epochs=50, imgsz=900, batch=8, name='pcb_model')  #Defining epochs, imagesize, batch and name of the model
model_path = f"{project_dir}/runs/detect/pcb_model"
print(f"Model saved at {model_path}")

##  Step 3: Evaluate the Model
trained_model = YOLO(model_path + '/weights/best.pt')
test_images = [
    '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/evaluation/ardmega.jpg',    #PATH to ardmega
    '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/evaluation/arduno.jpg',     #PATH to arduno
    '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/evaluation/rasppi.jpg'      #PATH to rasppi
]

# Analyze and save results
for img in test_images:
    results = trained_model.predict(img, save=True, save_dir=f"{model_path}/evaluation_results")
    print(f"Results for {img} saved to {model_path}/evaluation_results")

##  Step 4: Extract Ground Truth Labels (y_true)
labels_dir ='/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/test/labels' #PATH to Test_labels
test_image_names = ['evaluation_image1', 'evaluation_image2', 'evaluation_image3']

y_true = []
for image_name in test_image_names:
    label_file = os.path.join(labels_dir, f"{image_name}.txt")
    with open(label_file, 'r') as file:
        labels = [line.split()[0] for line in file.readlines()]
        y_true.extend(labels)
y_true = list(map(int, y_true))
print("Ground truth labels (y_true):", y_true)

# # Step 5: Extract Predicted Labels (y_pred)
y_pred = []
for img in test_images:
    results = trained_model.predict(img)
    predicted_classes = [int(box.cls) for box in results[0].boxes]
    y_pred.extend(predicted_classes)
print("Predicted labels (y_pred):", y_pred)

## Step 6: Generate and Visualize Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='viridis')
plt.title("Confusion Matrix")
plt.show()

##  Step 7: Visualize Training Metrics
metrics = trained_model.val()
precision = metrics['precision']
recall = metrics['recall']
plt.plot(recall, precision, label="Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()

# Additional insights for the Report
print("Key insights:")
print("- Use extracted image and saved evaluation results for the report.")
print("- Include precision-recall curve and confusion matrix in your submission.")


Extracted image saved to extracted_motherboard.JPEG
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=900, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pcb_model6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning /content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/train/labels.cache... 43 images, 501 backgrounds, 0 corrupt: 100%|██████████| 544/544 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/valid/labels.cache... 99 images, 6 backgrounds, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 26, len(boxes) = 18022. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        105      18022      0.112    0.00841     0.0601     0.0424



50 epochs completed in 0.524 hours.
Optimizer stripped from runs/detect/pcb_model6/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/pcb_model6/weights/best.pt, 6.3MB

Validating runs/detect/pcb_model6/weights/best.pt...
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,008,183 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:08<00:00,  1.28s/it]


                   all        105      18022      0.216     0.0124      0.114     0.0892
                Button         10         32          0          0          0          0
             Capacitor         99       6934      0.296     0.0299      0.161       0.11
             Connector         70        568       0.51      0.044      0.276      0.193
                 Diode          7         17          0          0          0          0
Electrolytic Capacitor         10         85          0          0          0          0
                    IC         90       1226      0.355    0.00897      0.182       0.14
              Inductor          9         30          0          0          0          0
                   Led         13         88          0          0          0          0
                  Pads         15        111          0          0          0          0
                  Pins          8        119       0.25     0.0084      0.127      0.102
              Resisto

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Project 3 Data/runs/detect/pcb_model/weights/best.pt'